In [1]:
import pandas as pd

# Carregando o dataset
data = pd.read_csv('Crime_Data_from_2020_to_Present.csv')


Criar as Colunas de Ano, Mês e Dia

In [2]:
data['year'] = data['DATE OCC'].str.split(' ').str[0].str.split('/').str[2].astype(int)
data['month'] = data['DATE OCC'].str.split(' ').str[0].str.split('/').str[1].astype(int)
data['day'] = data['DATE OCC'].str.split(' ').str[0].str.split('/').str[0].astype(int)


Agrupar os Dados para contar a ocorrência de crimes por área, mês e ano

In [4]:
data_process = data.groupby(['AREA', 'month', 'year'])['AREA'].count().reset_index(name='counts')
data_process.sort_values(by=['AREA', 'year', 'month'], inplace=True)


Modelo 1: Mês e Ano Discretizados, como variáveis de entrada.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Discretização de ano e mês
data_process['month'] = data_process['month'].astype('category')
data_process['year'] = data_process['year'].astype('category')

# Criar variáveis dummy para 'month' e 'year'
data_dummies = pd.get_dummies(data_process, columns=['month', 'year'])

# Dividir os dados
X = data_dummies.drop(['counts', 'AREA'], axis=1)
y = data_dummies['counts']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar e treinar o modelo
model1 = LinearRegression()
model1.fit(X_train, y_train)

# Fazer previsões
y_pred1 = model1.predict(X_test)

# Avaliar o modelo
mse1 = mean_squared_error(y_test, y_pred1)
print(f'MSE Modelo 1: {mse1}')


MSE Modelo 1: 5015.715365783411


Modelo 2: Mês e Ano Discretizados Trigonometricamente

In [7]:
import numpy as np

# Certifique-se de que as colunas 'month' e 'year' são do tipo inteiro
data_process['month'] = data_process['month'].astype(int)
data_process['year'] = data_process['year'].astype(int)

# Transformação trigonométrica
data_process['month_sin'] = np.sin(2 * np.pi * data_process['month'] / 12)
data_process['month_cos'] = np.cos(2 * np.pi * data_process['month'] / 12)
data_process['year_sin'] = np.sin(2 * np.pi * (data_process['year'] - data_process['year'].min()) / (data_process['year'].max() - data_process['year'].min()))
data_process['year_cos'] = np.cos(2 * np.pi * (data_process['year'] - data_process['year'].min()) / (data_process['year'].max() - data_process['year'].min()))

# Criar variáveis dummy para 'AREA'
data_dummies2 = pd.get_dummies(data_process, columns=['AREA'])

# Dividir os dados
X2 = data_dummies2[['month_sin', 'month_cos', 'year_sin', 'year_cos'] + [col for col in data_dummies2.columns if 'AREA_' in col]]
y2 = data_dummies2['counts']

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Criar e treinar o modelo
model2 = LinearRegression()
model2.fit(X_train2, y_train2)

# Fazer previsões
y_pred2 = model2.predict(X_test2)

# Avaliar o modelo
mse2 = mean_squared_error(y_test2, y_pred2)
print(f'MSE Modelo 2: {mse2}')


MSE Modelo 2: 5667.112080007615


Janelamento para 3 Meses
Para o modelo de janelamento, você deve criar um dataframe para cada área e considerar apenas as contagens dos 3 meses anteriores.

In [8]:
# Função para criar o janelamento
def create_lagged_features(df):
    for i in range(1, 4):
        df[f'counts_lag_{i}'] = df['counts'].shift(i)
    return df

# Criar janelas para cada área
data_lagged = data_process.groupby('AREA').apply(create_lagged_features).reset_index(drop=True)

# Remover valores nulos resultantes do janelamento
data_lagged.dropna(inplace=True)

# Dividir os dados
X3 = data_lagged[[f'counts_lag_{i}' for i in range(1, 4)] + ['month', 'year']]
y3 = data_lagged['counts']

X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.2, random_state=42)

# Criar e treinar o modelo
model3 = LinearRegression()
model3.fit(X_train3, y_train3)

# Fazer previsões
y_pred3 = model3.predict(X_test3)

# Avaliar o modelo
mse3 = mean_squared_error(y_test3, y_pred3)
print(f'MSE Modelo 3: {mse3}')


MSE Modelo 3: 1964.7047244653306


/tmp/ipykernel_60890/140604492.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_lagged = data_process.groupby('AREA').apply(create_lagged_features).reset_index(drop=True)


Comparar os Modelos

In [9]:
print(f'MSE Modelo 1: {mse1}')
print(f'MSE Modelo 2: {mse2}')
print(f'MSE Modelo 3: {mse3}')

# Avaliação final
if mse1 < mse2 and mse1 < mse3:
    print("Modelo 1 teve o melhor desempenho.")
elif mse2 < mse1 and mse2 < mse3:
    print("Modelo 2 teve o melhor desempenho.")
else:
    print("Modelo 3 teve o melhor desempenho.")


MSE Modelo 1: 5015.715365783411
MSE Modelo 2: 5667.112080007615
MSE Modelo 3: 1964.7047244653306
Modelo 3 teve o melhor desempenho.
